# Pengembangan Machine Learning Pipeline
- Nama: Baiq Ega Aulia
- Username dicoding: baiq_ega_9

- https://github.com/tensorflow/tfx/tree/master
- https://www.kaggle.com/datasets/nikhileswarkomati/suicide-watch/data
- https://colab.research.google.com/github/tensorflow/tfx/blob/master/docs/tutorials/tfx/penguin_simple.ipynb

# 1. Import Library and Dependency

In [ ]:
# conda create -n mlops-tfx python=3.9.15 -y
# conda activate mlops-tfx
# python -m pip install --upgrade pip
# pip install -r requirements.txt

# conda install ipykernel
# python -m ipykernel install --user --name=mlops-tfx --display-name "Python (mlops-tfx)"

In [1]:
# Importing necessary libraries
import os
import pandas as pd

# TensorFlow and TensorFlow Model Analysis
import tensorflow as tf
import tensorflow_model_analysis as tfma  # For model evaluation
from keras_tuner import RandomSearch # For hyperparameter tuning

# Importing TFX components
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator
from tfx.components import Transform, Trainer, Tuner, Evaluator, Pusher
from tfx.proto import example_gen_pb2, trainer_pb2, pusher_pb2

# TFX Interactive Context
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.orchestration.metadata import sqlite_metadata_connection_config

# Resolver for resolving inputs in pipelines
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy

# TFX standard artifact types
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing

# Requests and PrettyPrinter for external data handling and debugging
import requests
from pprint import PrettyPrinter

# 2. Set Variable

In [2]:
# Define pipeline and schema names
PIPELINE_NAME = "suicide-pipeline"  # Name of the main pipeline
SCHEMA_PIPELINE_NAME = "suicide-tfdv-schema"  # Name of the schema pipeline
MODEL_NAME = "suicide-model" # Name of the saved model

# Directory for storing generated artifacts
project_root = "C:/Users/Ega/Pictures/Dicoding/Membangun Sistem ML/Submission Pengembangan Machine Learning"
PIPELINE_ROOT = os.path.join(project_root, "baiq_ega_9-pipeline", PIPELINE_NAME)
METADATA_PATH = os.path.join(PIPELINE_ROOT, "metadata.db")  # SQLite metadata path
SERVING_MODEL_DIR = os.path.join("serving_model_dir", MODEL_NAME)

# Pipeline inputs
DATA_ROOT = "data"  # Root directory for input data
TRANSFORM_MODULE_FILE = "modules/suicide_transform.py"  # Transformation logic module
TRAINER_MODULE_FILE = "modules/suicide_trainer.py"  # Model training logic module
TUNER_MODULE_FILE = "modules/suicide_tuner.py"  # Hyperparameter tuning module
TRAINER_MODULE_FILE = "modules/suicide_trainer.py"  # Model training logic module (duplicate)

In [3]:
# Ensure necessary directories exist
modules_dir = os.path.join(project_root, "modules")
os.makedirs(modules_dir, exist_ok=True)
os.makedirs(os.path.dirname(METADATA_PATH), exist_ok=True)

In [4]:
# Create the TFX InteractiveContext with custom metadata path
interactive_context = InteractiveContext(
    pipeline_root=PIPELINE_ROOT,
    metadata_connection_config=sqlite_metadata_connection_config(METADATA_PATH)
)

# 3. Processing Dataset with Pandas

In [5]:
# 1. Load dataset
df = pd.read_csv('Suicide_Detection.csv')
print(f"Initial shape: {df.shape}")
print(f"Columns: {df.columns}")

Initial shape: (232074, 3)
Columns: Index(['Unnamed: 0', 'text', 'class'], dtype='object')


In [6]:
# 2. Map `class` to integers and rename column
label_mapping = {"suicide": 1, "non-suicide": 0}
df["class"] = df["class"].map(label_mapping)
df.rename(columns={"class": "is_suicide"}, inplace=True)

In [7]:
# 3. Drop unnecessary column (e.g., `Unnamed: 0` if it exists)
if "Unnamed: 0" in df.columns:
    df.drop(columns="Unnamed: 0", inplace=True)

In [8]:
# 4. Sampling 30,000 rows from each class
suicide_sample = df[df["is_suicide"] == 1].sample(5000, random_state=70)
non_suicide_sample = df[df["is_suicide"] == 0].sample(5000, random_state=70)

In [9]:
# 5. Combine the sampled data
balanced_df = pd.concat([suicide_sample, non_suicide_sample]).reset_index(drop=True)

# 6. Shuffle the combined dataset
balanced_df = balanced_df.sample(frac=1, random_state=70).reset_index(drop=True)

In [10]:
print(f"Initial shape: {balanced_df.shape}")
print(f"Columns: {balanced_df.columns}")

Initial shape: (10000, 2)
Columns: Index(['text', 'is_suicide'], dtype='object')


In [11]:
# 7. Save the processed data to a new CSV file
balanced_df.to_csv('data/Balanced_Suicide_Detection.csv', index=False)

In [12]:
# Print final dataset information
print(f"Final shape: {balanced_df.shape}")
print(balanced_df["is_suicide"].value_counts())
df.sample(5)

Final shape: (10000, 2)
1    5000
0    5000
Name: is_suicide, dtype: int64


,text,is_suicide
11244,I cannot handle thisI dont know what the fuck ...,1
64349,Awards are dumb Why would someone spend litera...,0
41708,"Suicidal thoughts.Hi, lately, I've been feelin...",1
162733,"I need to tell someone.Reddit, this was someth...",1
985,"Dear reddit, after taking a look at my life I ...",1


***Membuat Pipeline TFX Interaktif dengan Komponen-Komponen Utama***

Pipeline ini terdiri dari beberapa komponen utama *TFX* yang saling berhubungan untuk *membangun*, *melatih*, dan *mengevaluasi* model machine learning. Setiap komponen akan dijelaskan secara detail dengan contoh kode yang dilengkapi dengan **magic command** untuk membuat modul Python. *Pipeline* ini bersifat modular dan dapat disesuaikan dengan kebutuhan proyek Anda. Magic command seperti *%%writefile* mempermudah pembuatan modul khusus untuk komponen seperti *Transform*, *Trainer*, dan *Tuner*.

1. ```CsvExampleGen```
2. ```StatisticsGen```
3. ```SchemaGen```
4. ```ExampleValidator```
5. ```Transform```
6. ```Tuner```
7. ```Trainer```
8. ```Evaluator```
9. ```Pusher```

# 4. CsvExampleGen: Membaca dan membagi dataset
**Fungsi**: Membaca data dari file CSV dan membaginya menjadi subset `train` dan `eval`.  
- **Input**: File CSV di lokasi `DATA_ROOT`.  
- **Output**: Data dibagi menjadi 80% untuk `train` dan 20% untuk `eval`.  

In [13]:
# Konfigurasi ExampleGen
output = example_gen_pb2.Output( # Definisikan konfigurasi keluaran untuk pembagian data
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=9), # 90% data untuk pelatihan
        example_gen_pb2.SplitConfig.Split(name='eval',  hash_buckets=1) # 10% data untuk evaluasi
    ])
)

# Komponen CsvExampleGen untuk memproses data input dan menerapkan konfigurasi keluaran
example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output) # Gunakan DATA_ROOT sebagai input dan definisikan konfigurasi split

In [14]:
# Menjalankan komponen example_gen untuk memulai pipeline
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [15]:
# Verifikasi keluaran
print("CsvExampleGen outputs:", example_gen.outputs['examples'].get())

CsvExampleGen outputs: [Artifact(artifact: id: 1
type_id: 14
uri: "C:/Users/Ega/Pictures/Dicoding/Membangun Sistem ML/Submission Pengembangan Machine Learning\\baiq_ega_9-pipeline\\suicide-pipeline\\CsvExampleGen\\examples\\1"
properties {
  key: "split_names"
  value {
    string_value: "[\"train\", \"eval\"]"
  }
}
custom_properties {
  key: "file_format"
  value {
    string_value: "tfrecords_gzip"
  }
}
custom_properties {
  key: "input_fingerprint"
  value {
    string_value: "split:single_split,num_files:1,total_bytes:7080354,xor_checksum:1747410541,sum_checksum:1747410541"
  }
}
custom_properties {
  key: "payload_format"
  value {
    string_value: "FORMAT_TF_EXAMPLE"
  }
}
custom_properties {
  key: "span"
  value {
    int_value: 0
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.14.0"
  }
}
state: LIVE
, artifact_type: id: 14
name: "Examples"
properties {
  key: "span"
  value: INT
}
properties {
  key: "split_names"
  value: STRING
}
properties 

In [16]:
# Mendapatkan direktori keluaran TFRecord
output_uri = example_gen.outputs['examples'].get()[0].uri
print("TFRecord output directory:", output_uri)

TFRecord output directory: C:/Users/Ega/Pictures/Dicoding/Membangun Sistem ML/Submission Pengembangan Machine Learning\baiq_ega_9-pipeline\suicide-pipeline\CsvExampleGen\examples\1


In [17]:
# Menampilkan daftar file di direktori keluaran
print("Output files:", os.listdir(output_uri))

Output files: ['Split-eval', 'Split-train']


In [18]:
# Mendapatkan path split data untuk pelatihan
train_split_path = os.path.join(output_uri, "Split-train")
print("Train split files:", os.listdir(train_split_path))

Train split files: ['data_tfrecord-00000-of-00001.gz']


In [19]:
# Membaca sebagian data dari salah satu file
train_files = os.listdir(train_split_path)
if train_files:
    with open(os.path.join(train_split_path, train_files[0]), "rb") as f:
        print(f.read(100))

b'\x1f\x8b\x08\x00\x00\x00\x00\x00\x00\n\x9c\xbd\t\xbc$gY.\x9es\xe6\xcc\x84|\t (\xf7\xaf\xe2\xd5\xca w\x92\xd8\xd3\x93\x99\x84-\xb9q\x9cL&\x99\x83\x99d\xccL\x88a\x1b\xab\xbb\xab\xbb+\xa7\xbb\xaaSU}z:\\0   \xca*\xfb\xbex\x01qaQ@6\x11\xd9APv\x15\xd4 \x08\x01\x04DE\x08\xe0\xff'


In [20]:
# Membaca dan mem-parsing file TFRecord
# Mendapatkan daftar file TFRecord dengan tipe kompresi GZIP
tfrecord_files = tf.io.gfile.glob(f"{output_uri}/Split-train/*.gz")

# Membuat dataset dari file TFRecord
dataset_TFRecord = tf.data.TFRecordDataset(tfrecord_files, compression_type="GZIP")

# Menampilkan contoh data pertama dari TFRecord
for record in dataset_TFRecord.take(1):
    example = tf.train.Example.FromString(record.numpy())
    print(example)

features {
  feature {
    key: "is_suicide"
    value {
      int64_list {
        value: 1
      }
    }
  }
  feature {
    key: "text"
    value {
      bytes_list {
        value: "How can I get a hysterectomyFor people with dysphoria if you ever did anything like cut your chest tried to stab your uterus did the hospital take it out for you"
      }
    }
  }
}



# 5. Membuat Tahapan Data Validation

**Tahapan Data Validation terdiri dari tiga langkah utama:**
1. **Membuat Summary Statistic** untuk menganalisis data awal.
2. **Membuat Data Schema** untuk mendefinisikan struktur data.
3. **Validasi Dataset** untuk mengidentifikasi anomali berdasarkan statistik dan schema.

Pipeline ini memastikan bahwa dataset yang digunakan sesuai dengan ekspektasi sebelum dilanjutkan ke tahap berikutnya. 

## 5.1 StatisticsGen: Menghasilkan statistik deskriptif
**Fungsi**: Menghasilkan statistik deskriptif dari data untuk memahami distribusi dan kualitas data.  
- **Input**: Data yang dihasilkan oleh `CsvExampleGen`.  
- **Output**: Statistik fitur, termasuk distribusi nilai, rata-rata, standar deviasi, dan jumlah nilai kosong.  

In [21]:
# Komponen StatisticsGen untuk menghasilkan statistik dari data
statistics_gen = StatisticsGen(  # Membuat instance StatisticsGen
    examples=example_gen.outputs["examples"]  # Menggunakan output dari example_gen sebagai input
)

# Jalankan StatisticsGen menggunakan konteks interaktif
interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [26]:
# Menampilkan hasil statistik yang dihasilkan oleh StatisticsGen dalam format visual
interactive_context.show(statistics_gen.outputs['statistics'])

## 5.2 SchemaGen: Membuat schema data secara otomatis
**Fungsi**: Membuat schema data berdasarkan statistik dari `StatisticsGen`.  
- **Input**: Statistik dari `StatisticsGen`.  
- **Output**: Schema yang mendeskripsikan tipe data, batasan, dan distribusi nilai untuk setiap fitur.  

In [22]:
# Komponen SchemaGen untuk menghasilkan schema data secara otomatis
schema_gen = SchemaGen(  # Membuat instance SchemaGen
    statistics=statistics_gen.outputs["statistics"],  # Menggunakan output statistik dari StatisticsGen
)

# Jalankan SchemaGen untuk memproses statistik untuk menghasilkan schema data
interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [23]:
# Menampilkan schema data yang dihasilkan oleh SchemaGen dalam format interaktif
interactive_context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'is_suicide',INT,required,,-
'text',BYTES,required,,-


## 5.3 ExampleValidator: Mengidentifikasi anomali pada dataset
**Fungsi**: Memvalidasi dataset terhadap schema untuk mendeteksi anomali dan inkonsistensi.  
- **Input**: Statistik dari `StatisticsGen` dan schema dari `SchemaGen`.  
- **Output**: Laporan deteksi anomali pada data, seperti nilai kosong atau fitur yang tidak sesuai dengan schema.  

In [24]:
# Komponen ExampleValidator untuk memvalidasi data berdasarkan schema
example_validator = ExampleValidator(  # Membuat instance ExampleValidator
    statistics=statistics_gen.outputs['statistics'],  # Menggunakan statistik dari StatisticsGen
    schema=schema_gen.outputs['schema']  # Menggunakan schema yang dihasilkan oleh SchemaGen
)

# Jalankan ExampleValidator menggunakan konteks interaktif
interactive_context.run(example_validator)  # Memvalidasi data untuk mendeteksi anomali

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [25]:
# Menampilkan hasil deteksi anomali yang ditemukan oleh ExampleValidator
interactive_context.show(example_validator.outputs['anomalies'])  # Visualisasi hasil anomali dalam format interaktif

# 6. Transform: Data Processing

In [27]:
%%writefile {TRANSFORM_MODULE_FILE}

import tensorflow as tf
LABEL_KEY = "is_suicide"  # Kunci untuk label
FEATURE_KEY = "text"  # Kunci untuk fitur teks

def transformed_name(key):
    """Menambahkan '_xf' pada nama fitur atau label untuk menunjukkan transformasi."""
    return f"{key}_xf"

def preprocessing_fn(inputs):
    """
    Melakukan preprocessing pada fitur input menjadi fitur yang telah ditransformasi.
    
    Args:
        inputs: peta dari kunci fitur ke fitur mentah.
    
    Return:
        outputs: peta dari kunci fitur ke fitur yang telah ditransformasi.
    """
    outputs = {}
    
    # Mengonversi kalimat menjadi huruf kecil untuk normalisasi
    outputs[transformed_name(FEATURE_KEY)] = tf.strings.lower(inputs[FEATURE_KEY])
    # Mengonversi label menjadi tipe integer
    outputs[transformed_name(LABEL_KEY)]   = tf.cast(inputs[LABEL_KEY], tf.int64)
    
    return outputs

Overwriting modules/suicide_transform.py


In [28]:
# Komponen Transform untuk menerapkan transformasi data
transform = Transform(
    examples=example_gen.outputs["examples"],  # Menggunakan data dari ExampleGen
    schema=schema_gen.outputs["schema"],  # Menggunakan schema dari SchemaGen
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)  # File modul transformasi
)

# Jalankan Transform menggunakan konteks interaktif
interactive_context.run(transform)  # Melakukan transformasi pada data

INFO:tensorflow:Assets written to: C:/Users/Ega/Pictures/Dicoding/Membangun Sistem ML/Submission Pengembangan Machine Learning\baiq_ega_9-pipeline\suicide-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\3b7d184085d64b33be452024d7dd696d\assets


INFO:tensorflow:Assets written to: C:/Users/Ega/Pictures/Dicoding/Membangun Sistem ML/Submission Pengembangan Machine Learning\baiq_ega_9-pipeline\suicide-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\3b7d184085d64b33be452024d7dd696d\assets


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

# 7. Tuner: Hyperparameter Tuning

In [30]:
%%writefile {TUNER_MODULE_FILE}
import os
import tensorflow as tf
import tensorflow_transform as tft
import keras_tuner as kt
from tensorflow.keras import layers
from tfx.components.trainer.fn_args_utils import FnArgs
from keras_tuner.engine import base_tuner
from typing import NamedTuple, Dict, Text, Any
from keras_tuner import RandomSearch
from tfx.components import Tuner, Trainer
from tfx.proto import trainer_pb2

LABEL_KEY = 'is_suicide'
FEATURE_KEY = 'text'

# Utility function to rename transformed features
def transformed_name(key):
    return key + "_xf"

# Utility function to read gzip files
def gzip_reader_fn(filenames):
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

# Input function for creating datasets
def input_fn(file_pattern, tf_transform_output, num_epochs, batch_size=64):
    transform_feature_spec = tf_transform_output.transformed_feature_spec().copy()
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key=transformed_name(LABEL_KEY)
    )
    return dataset

VOCAB_SIZE = 5000
SEQUENCE_LENGTH = 1500

vectorize_layer = layers.TextVectorization(
    standardize='lower_and_strip_punctuation',
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=SEQUENCE_LENGTH
)

# Model builder for hyperparameter tuning
def model_builder(hp):
    """Build machine learning model"""
    embedding_dim = hp.Choice('embedding_dim', values=[32, 128])
    lstm_units = hp.Choice('lstm_units', values=[32, 128])
    dense_units = hp.Int('dense_units', min_value=32, max_value=128, step=16)
    dropout_rate = hp.Float('dropout_rate', min_value=0.1, max_value=0.3, step=0.1)
    learning_rate = hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-5])
    num_layers = hp.Int('num_layers', min_value=4, max_value=8, step=1)  # Added num_layers here

    inputs = tf.keras.Input(shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string)
    reshaped_narrative = tf.reshape(inputs, [-1])
    x = vectorize_layer(reshaped_narrative)
    x = layers.Embedding(VOCAB_SIZE, embedding_dim, name='embedding')(x)
    x = layers.Bidirectional(layers.LSTM(lstm_units))(x)

    for _ in range(num_layers):  # Use num_layers here
        x = layers.Dense(dense_units, activation='relu')(x)
    x = layers.Dropout(dropout_rate)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
        optimizer=tf.keras.optimizers.Adam(learning_rate),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )
    return model

TunerFnResult = NamedTuple('TunerFnResult', [
    ('tuner', base_tuner.BaseTuner),
    ('fit_kwargs', Dict[Text, Any]),
])

early_stop_callback = tf.keras.callbacks.EarlyStopping(
    monitor  = 'val_binary_accuracy',
    mode     = 'max',
    verbose  = 1,
    patience = 2
)

# Tuner function
def tuner_fn(fn_args: FnArgs):
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    train_set = input_fn(fn_args.train_files[0], tf_transform_output, num_epochs=3, batch_size=64)
    val_set = input_fn(fn_args.eval_files[0], tf_transform_output, num_epochs=3, batch_size=64)

    vectorize_layer.adapt(
        [j[0].numpy()[0] for j in [
            i[0][transformed_name(FEATURE_KEY)] for i in list(train_set)
        ]]
    )

    model_tuner = RandomSearch(
        hypermodel=model_builder,
        objective=kt.Objective('val_binary_accuracy', direction='max'),
        max_trials=4,
        executions_per_trial=2,
        directory=fn_args.working_dir,
        project_name='suicide_text_random_tuner',
    )

    return TunerFnResult(
        tuner=model_tuner,
        fit_kwargs={
            'x': train_set,
            'validation_data': val_set,
            'steps_per_epoch': fn_args.train_steps,
            'validation_steps': fn_args.eval_steps,
            'callbacks': [
                tf.keras.callbacks.EarlyStopping(
                    monitor='val_binary_accuracy',
                    mode='max',
                    patience=2,
                    verbose=1
                )
            ]
        }
    )


Overwriting modules/suicide_tuner.py


In [31]:
# Pipeline Tuner Component
tuner = Tuner(
    module_file     = os.path.abspath(TUNER_MODULE_FILE),
    examples        = transform.outputs['transformed_examples'],
    transform_graph = transform.outputs['transform_graph'],
    schema          = schema_gen.outputs['schema'],
    train_args      = trainer_pb2.TrainArgs(splits=['train']),
    eval_args       = trainer_pb2.EvalArgs(splits=['eval'])
)

interactive_context.run(tuner)

Trial 4 Complete [00h 09m 08s]
val_binary_accuracy: 0.9163244366645813

Best val_binary_accuracy So Far: 0.9250513315200806
Total elapsed time: 01h 14m 49s
Results summary
Results in C:/Users/Ega/Pictures/Dicoding/Membangun Sistem ML/Submission Pengembangan Machine Learning\baiq_ega_9-pipeline\suicide-pipeline\.temp\6\suicide_text_random_tuner
Showing 10 best trials
Objective(name="val_binary_accuracy", direction="max")

Trial 0 summary
Hyperparameters:
embedding_dim: 32
lstm_units: 32
dense_units: 32
dropout_rate: 0.2
learning_rate: 0.001
num_layers: 5
Score: 0.9250513315200806

Trial 1 summary
Hyperparameters:
embedding_dim: 128
lstm_units: 32
dense_units: 96
dropout_rate: 0.2
learning_rate: 0.0001
num_layers: 8
Score: 0.9250513315200806

Trial 3 summary
Hyperparameters:
embedding_dim: 32
lstm_units: 32
dense_units: 48
dropout_rate: 0.2
learning_rate: 0.001
num_layers: 7
Score: 0.9163244366645813

Trial 2 summary
Hyperparameters:
embedding_dim: 32
lstm_units: 32
dense_units: 80
dropo

ExecutionResult(
    component_id: Tuner
    execution_id: 6
    outputs:
        best_hyperparameters: OutputChannel(artifact_type=HyperParameters, producer_component_id=Tuner, output_key=best_hyperparameters, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        tuner_results: OutputChannel(artifact_type=TunerResults, producer_component_id=Tuner, output_key=tuner_results, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

# 8. Trainer: Model Development

In [32]:
%%writefile {TRAINER_MODULE_FILE}
import tensorflow as tf
import tensorflow_transform as tft 
from tensorflow.keras import layers
import os  
import tensorflow_hub as hub
from tfx.components.trainer.fn_args_utils import FnArgs
 
LABEL_KEY = "is_suicide"
FEATURE_KEY = "text"
 
def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"
 
def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')
 
 
def input_fn(file_pattern, 
             tf_transform_output,
             num_epochs,
             batch_size=64)->tf.data.Dataset:
    """Get post_tranform feature & create batches of data"""
    
    # Get post_transform feature spec
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy())
    
    # create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key = transformed_name(LABEL_KEY))
    
    # # Add dataset repeat operation
    # dataset = dataset.repeat()  # Ensure dataset repeats indefinitely or for specified epochs
    return dataset
 

# Vocabulary size and number of words in a sequence.
VOCAB_SIZE = 10000
SEQUENCE_LENGTH = 120
embedding_dim = 16

vectorize_layer = layers.TextVectorization(
    standardize="lower_and_strip_punctuation",
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=SEQUENCE_LENGTH)
 

def model_builder(hp):
    """Build machine learning model"""
    inputs = tf.keras.Input(shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string)
    
    reshaped_narrative = tf.reshape(inputs, [-1])
    x = vectorize_layer(reshaped_narrative)
    x = layers.Embedding(VOCAB_SIZE, hp['embedding_dim'], name='embedding')(x)
    x = layers.Bidirectional(layers.LSTM(hp['lstm_units']))(x)
    for _ in range(hp['num_layers']):
        x = layers.Dense(hp['dense_units'], activation='relu')(x)
    x = layers.Dropout(hp['dropout_rate'])(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    
    model = tf.keras.Model(inputs = inputs, outputs = outputs)
    model.compile(
        loss      = tf.keras.losses.BinaryCrossentropy(from_logits=False),
        optimizer = tf.keras.optimizers.Adam(hp['learning_rate']),
        metrics   = [tf.keras.metrics.BinaryAccuracy()]
    )
    
    model.summary()
    return model
 
 
def _get_serve_tf_examples_fn(model, tf_transform_output):
    
    model.tft_layer = tf_transform_output.transform_features_layer()
    
    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(LABEL_KEY)
        
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        transformed_features = model.tft_layer(parsed_features)
        
        # get predictions using the transformed features
        return model(transformed_features)
        
    return serve_tf_examples_fn
    
# Trainer function
def run_fn(fn_args: FnArgs):
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    train_set = input_fn(fn_args.train_files, tf_transform_output, num_epochs=2, batch_size=64)
    val_set = input_fn(fn_args.eval_files, tf_transform_output, num_epochs=2, batch_size=64)

    vectorize_layer.adapt(
        [j[0].numpy()[0] for j in [
            i[0][transformed_name(FEATURE_KEY)] for i in list(train_set)
        ]]
    )

    hp = fn_args.hyperparameters['values']
    model = model_builder(hp)

    callbacks = [
        tf.keras.callbacks.TensorBoard(log_dir=os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')),
        tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=10, mode='max', verbose=1),
        tf.keras.callbacks.ModelCheckpoint(
            filepath=fn_args.serving_model_dir,
            monitor='val_binary_accuracy',
            mode='max',
            save_best_only=True,
            verbose=1
        )
    ]

    model.fit(
        x=train_set,
        validation_data=val_set,
        epochs=2,
        steps_per_epoch=fn_args.train_steps,
        validation_steps=fn_args.eval_steps,
        callbacks=callbacks
    )

    signatures = {
        'serving_default': tf.function(
            model
        ).get_concrete_function(
            tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
        )
    }

    model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

Overwriting modules/suicide_trainer.py


In [33]:
trainer = Trainer(
    module_file     = os.path.abspath(TRAINER_MODULE_FILE),
    examples        = transform.outputs['transformed_examples'],
    transform_graph = transform.outputs['transform_graph'],
    schema          = schema_gen.outputs['schema'],
    hyperparameters = tuner.outputs['best_hyperparameters'],
    train_args      = trainer_pb2.TrainArgs(splits=['train']),
    eval_args       = trainer_pb2.EvalArgs(splits=['eval'])
)

interactive_context.run(trainer)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_xf (InputLayer)        [(None, 1)]               0         
                                                                 
 tf.reshape_1 (TFOpLambda)   (None,)                   0         
                                                                 
 text_vectorization (TextVe  (None, 120)               0         
 ctorization)                                                    
                                                                 
 embedding (Embedding)       (None, 120, 32)           320000    
                                                                 
 bidirectional_1 (Bidirecti  (None, 64)                16640     
 onal)                                                           
                                                                 
 dense_8 (Dense)             (None, 32)                2080

INFO:tensorflow:Assets written to: C:/Users/Ega/Pictures/Dicoding/Membangun Sistem ML/Submission Pengembangan Machine Learning\baiq_ega_9-pipeline\suicide-pipeline\Trainer\model\7\Format-Serving\assets


283/283 [==============================] - 24s 73ms/step - loss: 0.3465 - binary_accuracy: 0.8495 - val_loss: 0.2165 - val_binary_accuracy: 0.9138
Epoch 2/2
283/283 [==============================] - ETA: 0s - loss: 0.1552 - binary_accuracy: 0.9469
Epoch 2: val_binary_accuracy did not improve from 0.91376
283/283 [==============================] - 13s 47ms/step - loss: 0.1552 - binary_accuracy: 0.9469 - val_loss: 0.2364 - val_binary_accuracy: 0.9045
INFO:tensorflow:Assets written to: C:/Users/Ega/Pictures/Dicoding/Membangun Sistem ML/Submission Pengembangan Machine Learning\baiq_ega_9-pipeline\suicide-pipeline\Trainer\model\7\Format-Serving\assets


INFO:tensorflow:Assets written to: C:/Users/Ega/Pictures/Dicoding/Membangun Sistem ML/Submission Pengembangan Machine Learning\baiq_ega_9-pipeline\suicide-pipeline\Trainer\model\7\Format-Serving\assets


ExecutionResult(
    component_id: Trainer
    execution_id: 7
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

# 9. Model Analysis and Validation

## 9.1 Resolver

In [34]:

model_resolver = Resolver(
    strategy_class= LatestBlessedModelStrategy,
    model = Channel(type=Model),
    model_blessing = Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')
 
interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 8
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

## 9.2 Evaluator

In [35]:
eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='is_suicide')],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[
            
            tfma.MetricConfig(class_name='ExampleCount'),
            tfma.MetricConfig(class_name='AUC'),
            tfma.MetricConfig(class_name='FalsePositives'),
            tfma.MetricConfig(class_name='TruePositives'),
            tfma.MetricConfig(class_name='FalseNegatives'),
            tfma.MetricConfig(class_name='TrueNegatives'),
            tfma.MetricConfig(class_name='BinaryAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value':0.5}),
                    change_threshold=tfma.GenericChangeThreshold(
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute={'value':0.0001})
                    )
            )
        ])
    ]
 
)

In [36]:
evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)
 
interactive_context.run(evaluator)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 9
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [37]:
# Visualize the evaluation results
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(
    tfma_result
)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'binary_accu…

In [38]:
tfma_result

EvalResult(slicing_metrics=[((), {'': {'': {'binary_accuracy': {'doubleValue': 0.8973305954825462}, 'loss': {'doubleValue': 0.24879151582717896}, 'example_count': {'doubleValue': 974.0}, 'auc': {'doubleValue': 0.9646561181434598}, 'false_positives': {'doubleValue': 61.0}, 'true_positives': {'doubleValue': 435.0}, 'false_negatives': {'doubleValue': 39.0}, 'true_negatives': {'doubleValue': 439.0}}}})], plots=[((), None)], attributions=[((), None)], config=model_specs {
  label_key: "is_suicide"
}
slicing_specs {
}
metrics_specs {
  metrics {
    class_name: "ExampleCount"
  }
  metrics {
    class_name: "AUC"
  }
  metrics {
    class_name: "FalsePositives"
  }
  metrics {
    class_name: "TruePositives"
  }
  metrics {
    class_name: "FalseNegatives"
  }
  metrics {
    class_name: "TrueNegatives"
  }
  metrics {
    class_name: "BinaryAccuracy"
    threshold {
      value_threshold {
        lower_bound {
          value: 0.5
        }
      }
    }
  }
  model_names: ""
}
, data_loca

# 10. Pusher: Model Deployment and Export

In [39]:
pusher = Pusher(
model=trainer.outputs['model'],
model_blessing=evaluator.outputs['blessing'],
push_destination=pusher_pb2.PushDestination(
    filesystem=pusher_pb2.PushDestination.Filesystem(
        base_directory=SERVING_MODEL_DIR))
 
)
 
interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 10
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

## 10.1 Check End-Point Model Yang Digunakan Oleh TF-Serving

In [1]:
from pprint import PrettyPrinter
import requests

pp = PrettyPrinter()
pp.pprint(requests.get("http://localhost:8501/v1/models/suicide-model").json())


{'model_version_status': [{'state': 'AVAILABLE',
                           'status': {'error_code': 'OK', 'error_message': ''},
                           'version': '1747415592'}]}
